In [1]:
# Install necessary libraries
!pip install transformers
#!pip install -U transformers
#!pip install sentencepiece

import os
import urllib.request
import zipfile
import tensorflow as tf
from transformers import MarianTokenizer, MarianMTModel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.8 MB/s eta 0:00:00


In [2]:
# Define constants
model_name = "Helsinki-NLP/opus-mt-es-en"
dataset_url = "http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
max_length = 128
batch_size = 32
num_epochs = 5
learning_rate = 1e-4

In [3]:
# Download and extract data
dataset_dir = "spa-eng"
file_name = "spa-eng.zip"

In [4]:
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

urllib.request.urlretrieve(dataset_url, os.path.join(dataset_dir, file_name))
with zipfile.ZipFile(os.path.join(dataset_dir, file_name), 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)


In [5]:
# Load data
file_path = os.path.join("spa-eng", "spa-eng", "spa.txt")
lines = open(file_path, encoding='utf-8').read().strip().split('\n')
train_data = [(line.split('\t')[0], line.split('\t')[1]) for line in lines]
source_texts, target_texts = zip(*train_data)

In [6]:
# Initialize tokenizer and model
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

ImportError: ignored

In [7]:
# Preprocess data
input_encodings = tokenizer(source_texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')
target_encodings = tokenizer(target_texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')

inputs = {key: tf.convert_to_tensor(input_encodings[key]) for key in input_encodings}
targets = tf.convert_to_tensor(target_encodings['input_ids'])

NameError: ignored

In [ ]:
# Build train dataset
train_dataset = tf.data.Dataset.from_tensor_slices((inputs, targets))
train_dataset = train_dataset.shuffle(len(inputs)).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)


In [ ]:
# Define training step
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        logits = model(**inputs, decoder_input_ids=targets).logits  # Pass inputs as **kwargs
        loss = loss_fn(targets, logits)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [ ]:
# Training parameters
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
# Training loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    total_loss = 0
    total_batches = 0
    for batch, (inputs, targets) in enumerate(train_dataset):
        loss = train_step(inputs, targets)  # Use the regular train step
        total_loss += loss.numpy().item()
        total_batches = batch + 1
        if batch % 100 == 0:
            print(f"Batch {batch}, Loss: {loss.numpy():.4f}")
    print(f"Epoch Loss: {total_loss / total_batches:.4f}")

In [ ]:
# Translate a source text to the target language
def translate_text(input_text, tokenizer, model, max_length=128):
    input_encoding = tokenizer(input_text, padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')
    input_ids = tf.convert_to_tensor(input_encoding['input_ids'])
    translation_ids = model.generate(input_ids)
    translation_text = tokenizer.batch_decode(translation_ids, skip_special_tokens=True)[0]
    return translation_text

# Example source text for translation
source_text = "Hello, how are you?"

# Perform translation
translated_text = translate_text(source_text, tokenizer, model)

# Print the translated text
print(f"Source Text: {source_text}")
print(f"Translated Text: {translated_text}")
